In [2]:
TECH_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Tech'
CULTURE_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Culture'
Finance_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Finance'
Medical_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Medical'
politics_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Politics'
Religion_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Religion'
Sports_FOLDER_PATH = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\raw\Sports'

DOC_DIRECTORY = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\processed\all_split.pkl'
Sample_DIRECTORY = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\processed\sys_sample.pkl'


# Specify the directory where you want to save the vector database
PERSIST_DIRECTORY = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\processed\sample_vector_db2'

In [28]:
import os
import pickle
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from tqdm import tqdm  # Import tqdm for the progress bar





# Functions

In [4]:

def read_text_files_in_folder(folders_paths: list):
    """
    Reads text files from multiple folders and returns a list of Document objects.

    Args:
        folders_paths (list): A list of folder paths containing text files.

    Returns:
        all_docs (list): A list of Document objects, where each document contains
                         the text content of a file and its metadata with the file path.

    The function iterates over each folder in the provided list of folder paths. It reads
    all `.txt` files within each folder, creates a Document object for each file with the
    text content and the file's metadata, and appends it to the list of documents.
    A progress bar is displayed for each folder to indicate the processing status.
    """
    all_docs = []
    for folder_path in folders_paths:
        # Use tqdm to add a progress bar around the list of files
        for file_name in tqdm(os.listdir(folder_path), desc=f"Processing {os.path.basename(folder_path)}"):
            if file_name.endswith('.txt'):
                file_path = os.path.join(folder_path, file_name)
                with open(file_path, 'r', encoding='utf-8') as file:
                    text = file.read()
                    # Create a Document object for each text
                    doc = Document(page_content=text, metadata={"source": file_path})
                    all_docs.append(doc)
    return all_docs

# ----------------------------------------------------------------



def create_chroma_batches(documents, embedding_model, persist_directory, batch_size=5000):
    """
    Processes documents in batches and creates a Chroma vector store with embeddings.

    Args:
        documents (list): A list of Document objects to be processed.
        embedding_model (SentenceTransformer): The embedding model used to generate embeddings for the documents.
        persist_directory (str): The directory where the Chroma vector store will be persisted.
        batch_size (int, optional): The maximum number of documents to process in each batch. Default is 5000.

    Returns:
        chroma_instance (Chroma): The Chroma vector store instance containing the embedded documents.

    The function splits the documents into smaller batches and processes each batch to create
    a Chroma vector store with embeddings. A progress bar is used to show the progress of batch processing.
    If a Chroma instance does not exist, it is created with the first batch. Subsequent batches are added
    to the existing Chroma instance.
    """
    chroma_instance = None

    # Initialize the progress bar
    num_batches = len(documents) // batch_size + int(len(documents) % batch_size != 0)
    with tqdm(total=num_batches, desc="Processing batches", unit="batch") as pbar:
        # Process documents in batches
        for i in range(0, len(documents), batch_size):
            batch = documents[i:i + batch_size]
            
            if chroma_instance is None:
                # Create a new Chroma instance with the first batch
                chroma_instance = Chroma.from_documents(
                    documents=batch,
                    embedding=embedding_model,
                    persist_directory=persist_directory
                )
            else:
                # Add the batch to the existing Chroma instance
                chroma_instance.add_documents(documents=batch)
            
            # Update the progress bar
            pbar.update(1)

    return chroma_instance


all-MiniLM-L12-v2

- Average Performance: 59.76
- Speed: 7500 (fast)
- Model Size: 120 MB
- Strengths: High average performance with a good balance between speed and model size.

In [9]:
# Initialize the model 
model = SentenceTransformer('all-MiniLM-L12-v2')

class embedding:
    def __init__(self):
        self.model = model
    def embed_documents(self , docs):
        embeddings = self.model.encode(docs)
        return embeddings.tolist()
    
    def embed_query(self , query):
        return self.model.encode(query).tolist()

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


------

# Load the data

### Because the data already split into small pieces and each base in different file so we don't need to chunk them

In [6]:
folders_paths = [TECH_FOLDER_PATH ,
                 CULTURE_FOLDER_PATH , 
                 Finance_FOLDER_PATH , 
                 Medical_FOLDER_PATH,
                 politics_FOLDER_PATH ,
                 Sports_FOLDER_PATH, 
                 Religion_FOLDER_PATH 
                 ]
folders_paths

['D:\\Projects\\RAG-Powered-Arabic-AI-Assistant\\data\\raw\\Tech',
 'D:\\Projects\\RAG-Powered-Arabic-AI-Assistant\\data\\raw\\Culture',
 'D:\\Projects\\RAG-Powered-Arabic-AI-Assistant\\data\\raw\\Finance',
 'D:\\Projects\\RAG-Powered-Arabic-AI-Assistant\\data\\raw\\Medical',
 'D:\\Projects\\RAG-Powered-Arabic-AI-Assistant\\data\\raw\\Politics',
 'D:\\Projects\\RAG-Powered-Arabic-AI-Assistant\\data\\raw\\Sports',
 'D:\\Projects\\RAG-Powered-Arabic-AI-Assistant\\data\\raw\\Religion']

In [7]:
all_split =  read_text_files_in_folder(folders_paths)

Processing Religion: 100%|██████████| 6500/6500 [00:06<00:00, 1034.93it/s]


### store all doc

In [8]:
import pickle

# Save the all_split list to a file
with open(DOC_DIRECTORY, 'wb') as f:
    pickle.dump(all_split, f)

In [7]:


# Load the all_split list from the file
with open(DOC_DIRECTORY, 'rb') as f:
    all_split = pickle.load(f)

In [8]:
print(f"Total documents: {len(all_split)}")

Total documents: 45500


In [7]:
6499 * 7

45493

### for The limited resources and time i will use Systematic sampling to take samples from all categories

In [15]:
sys_sampling =  all_split[: : 6]

In [20]:
import pickle

# Save the all_split list to a file
with open(Sample_DIRECTORY, 'wb') as f:
    pickle.dump(sys_sampling, f)

In [12]:
with open(Sample_DIRECTORY, 'rb') as f:
    sys_sampling = pickle.load(f)

# Embedding

In [10]:
embed_model = embedding()

len(embed_model.embed_query('مرحبا بك'))

384

# Vector DataBase

### Chroma maximum batch size 41666 so we will take 41666 record only 

In [12]:
all_split = sys_sampling

In [13]:
len(all_split)

7584

In [17]:
# Process your documents in smaller batches with a progress bar
vector_data = create_chroma_batches(
    documents=all_split,
    embedding_model=embed_model,
    persist_directory=PERSIST_DIRECTORY,
    batch_size=20  # Set a batch size within the limit
)


Processing batches: 100%|██████████| 380/380 [13:55<00:00,  2.20s/batch]


In [19]:
reteiever = vector_data.as_retriever(search_type = 'similarity' , search_kwargs= {'k' : 2})
all_split[1].page_content

'دبي - مدحت السويفي:قال أحمد الخطيب، المؤسس والرئيس التنفيذي لموقع "ماركة في آي بي" في تصريحات خاصة ل"الخليج"، إنّ الشركة تعتبر من أكبر الشركات في مجال بيع الأزياء والإكسسوارات والعلامات التجارية الفاخرة عبر الانترنت، حيث تستحوذ الشركة على نسبة تقدر ب 50% عبر مبيعات "الأون لاين" لهذا القطاع تحديداً. أشار الخطيب، إلى أن إجمالي حجم قطاع تجارة التجزئة في منطقة الشرق الأوسط يقدر ب 425 مليار دولار، إلا أن حصة التسويق الالكتروني بالمنطقة منها لا يتجاوز ال 10 مليارات دولار بنسبة تتحرك في دائرة ال 2%، في الوقت الذي تبلغ فيه حصة هذه السوق الإلكترونية في أسواق أوروبا وأمريكا بنسبة تتراوح ما بين 15 و25%، وبحسب دراسات لمؤسسات دولية موثقة عن القطاع، وهو ما يشير إلى أن السوق ما زالت في طور التكوين وأن المستقبل واعد أمام هذا القطاع .وأوضح الخطيب، أن الشركة تحقق مبيعات مطردة من فترة لأخرى، فمنذ بداية العام الجاري حققت الشركة زيادة في مبيعات تقدر ب 100%، لافتاً إلى أن الشركة تضاعف مبيعاتها كل أربعة أشهر اعتماداً على اتساع قاعدة العملاء الذين يرغبون بالتسوق الإلكتروني، إضافة للانتشار الكبير للهواتف الذك

In [22]:
# Later, you can load the database like this:
loaded_vector_data = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function = embed_model
)
reteiever = loaded_vector_data.as_retriever(search_type = 'similarity' , search_kwargs= {'k' : 3})
sys_sampling[8].page_content

'أقر مجلس إدارة شركة اتصالات دي بي تغيير مسمى شركة سوان تيليكوم التي تأسست بموجب اتفاقية شراكة بين كل من اتصالات وشركة دي بي للتطوير العقاري ليصبح مسمى الشركة الجديد اتصالات دي بي تيليكوم الهند الخاصة المحدودة، وتتخذ من مومباي في الهند مقراً لها . كانت شركة اتصالات دي بي تيليكوم الهند الخاصة المحدودة قد حصلت على رخصة تزويد خدمات الاتصال العالمية الموحدة في 15 دائرة هندية هي اندرا براديش ودلهي وكوجارات وهاريانا كارنكاتا وكيرلا ومهراشترا ومومباي والبنجاب وراجستان وتاميل نادو(بما فيها تشيناي) واوتاربراديش (شرق) واتار براديش (غرب) ومديا يراديش وبيهار .وستتمكن الشركة بفضل هذه الرخصة من توفير مجموعة شاملة من خدمات الاتصالات لما يزيد على 900 مليون شخص في مختلف مناطق الهند . وتتضمن الخدمات الحلول الهاتفية للمكالمات المحلية والدولية ومجموعة كبيرة من خدمات الدفع المسبق وخدمات الدفع المفوترة وخدمات التجوال المحلية والدولية ومجموعة كبيرة من خدمات القيمة المضافة التي تتضمن البريد الصوتي وخدمات اديويتكس وخدمات التواصل عبر الفيديو والفيديوتكس والبريد الإلكتروني وخدمات التواصل بين المجموعات وبموجب الت

In [23]:
reteiever.invoke("أقر مجلس إدارة شركة اتصالات دي بي تغيير مسمى شركة سوان تيليكوم التي تأسست بموجب اتفاقية شراكة بين كل من ")


[Document(metadata={'source': 'D:\\Projects\\RAG-Powered-Arabic-AI-Assistant\\data\\raw\\Tech\\0048.txt'}, page_content='أقر مجلس إدارة شركة اتصالات دي بي تغيير مسمى شركة سوان تيليكوم التي تأسست بموجب اتفاقية شراكة بين كل من اتصالات وشركة دي بي للتطوير العقاري ليصبح مسمى الشركة الجديد اتصالات دي بي تيليكوم الهند الخاصة المحدودة، وتتخذ من مومباي في الهند مقراً لها . كانت شركة اتصالات دي بي تيليكوم الهند الخاصة المحدودة قد حصلت على رخصة تزويد خدمات الاتصال العالمية الموحدة في 15 دائرة هندية هي اندرا براديش ودلهي وكوجارات وهاريانا كارنكاتا وكيرلا ومهراشترا ومومباي والبنجاب وراجستان وتاميل نادو(بما فيها تشيناي) واوتاربراديش (شرق) واتار براديش (غرب) ومديا يراديش وبيهار .وستتمكن الشركة بفضل هذه الرخصة من توفير مجموعة شاملة من خدمات الاتصالات لما يزيد على 900 مليون شخص في مختلف مناطق الهند . وتتضمن الخدمات الحلول الهاتفية للمكالمات المحلية والدولية ومجموعة كبيرة من خدمات الدفع المسبق وخدمات الدفع المفوترة وخدمات التجوال المحلية والدولية ومجموعة كبيرة من خدمات القيمة المضافة التي تتضمن البريد 